In [1326]:
import pandas as pd
import numpy as np

In [1327]:
filename = "../.gitignore/data_a (Florent).csv"
df = pd.read_csv(filename, skipinitialspace=True) #Open the csv file with no space before and after the values

In [1328]:
print(df.columns)
print(f"rows : {df.shape[0]}\ncolumns : {df.shape[1]}")

Index(['Unnamed: 0', 'To rent', 'To sell', 'Price', 'Number of rooms',
       'Living Area', 'Fully equipped kitchen', 'Furnished', 'Open fire',
       'Terrace', 'Area of the terrace', 'Garden', 'Area of the garden',
       'Surface of the land', 'Surface area of the plot of land',
       'Number of facades', 'Swimming pool', 'State of the building',
       'zipcode', 'type'],
      dtype='object')
rows : 78192
columns : 20


In [1329]:
# Drop the unnamed:0 column
df = df.drop(columns='Unnamed: 0')
df.head(2)
df['type'].values

array(['Studio', 'Appartement', 'Maison', ..., 'Appartement', 'Bungalow',
       'Duplex'], dtype=object)

In [1343]:
#TEST
#Taking all the rent buildings directly from the source
df_rent = df.query("`To rent` == True" )

#Taking only the appartments and Rez-de-chaussée
df_rent_appart = df.query("type == 'Appartement' | type == 'Rez-de-chaussée'")

del df_rent_appart['Number of rooms'] #Testing adding it and deleting the rows with the missing value df_rent_appart = df_rent_appart.dropna(subset='Numbers of rooms')
df_rent_appart = df_rent_appart.dropna(subset = 'Living Area')
df_rent_appart = df_rent_appart.dropna(subset = 'State of the building')
print(f"There are {df_rent_appart.shape[0]} rows left")
#How many missing values are there ?
df_rent_appart.isnull().sum()



#Creating the column "Province"

df_rent_appart[(df_rent_appart["zipcode"] >= 1000) & (df_rent_appart["zipcode"] <= 1299)]

Province = []
for zipcode in df_rent_appart['zipcode']:
    if (zipcode >= 1000) & (zipcode <= 1299): Province.append('Brussels')
    elif (zipcode >= 1300) & (zipcode <= 1499): Province.append('Brabant wallon')
    elif (zipcode >= 2000) & (zipcode <= 2999): Province.append('Anvers')
    elif (zipcode >= 3500) & (zipcode <= 3999): Province.append('Limbourg')
    elif (zipcode >= 4000) & (zipcode <= 4999): Province.append('Liege')
    elif (zipcode >= 5000) & (zipcode <= 5680): Province.append('Namur')
    elif (zipcode >= 6600) & (zipcode <= 6999): Province.append('Luxembourg')
    elif (zipcode >= 8000) & (zipcode <= 8999): Province.append('Flandre occiendentale')
    elif (zipcode >= 9000) & (zipcode <= 9999): Province.append('Flandre orientale')
    elif (zipcode >= 1500) & (zipcode <= 1999) or (zipcode >= 3000) & (zipcode <= 3499): Province.append('Brabant flamand')
    elif (zipcode >= 6000) & (zipcode <= 6599) or (zipcode >= 7000) & (zipcode <= 7999): Province.append('Hainaut')

df_rent_appart['Province'] = Province
del df_rent_appart['zipcode'] #Not needed anymore
del df_rent_appart['type']


print(df_rent_appart.columns)





df_rent_appart = pd.get_dummies(data = df_rent_appart, columns = ['Province', 'State of the building'])









#Modelization part for renting appart in Belgium with LinearRegression

#Defining the features and the target

print(df_rent_appart.columns)

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part

regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")



There are 17134 rows left
Index(['To rent', 'To sell', 'Price', 'Living Area', 'Open fire', 'Terrace',
       'Garden', 'Swimming pool', 'State of the building', 'Province'],
      dtype='object')
Index(['To rent', 'To sell', 'Price', 'Living Area', 'Open fire', 'Terrace',
       'Garden', 'Swimming pool', 'Province_Anvers',
       'Province_Brabant flamand', 'Province_Brabant wallon',
       'Province_Brussels', 'Province_Flandre occiendentale',
       'Province_Flandre orientale', 'Province_Hainaut', 'Province_Liege',
       'Province_Limbourg', 'Province_Luxembourg', 'Province_Namur',
       'State of the building_\nBon\n',
       'State of the building_\nExcellentétat\n',
       'State of the building_\nFraîchementrénové\n',
       'State of the building_\nÀrafraîchir\n',
       'State of the building_\nÀrestaurer\n',
       'State of the building_\nÀrénover\n'],
      dtype='object')
The score of the model for training part : 0.598008063856774
The score of the model for testing pa

In [1331]:
#Deleting the 3992 rows on 78192 who have not the rent or sell info and who haven't any price
df = df.dropna(subset=['Price'])

#Deleting the column "surface of the land", "Surface area of the plot of land"
df = df.drop(columns=['Surface of the land', 'Surface area of the plot of land'])

#Deleting the number of facades column and Area of the garden column and Area of the terrace

df = df.drop(columns=['Number of facades', 'Area of the garden', 'Area of the terrace'])

#Deleting the fully equipped kitchen, furnished and number of rooms columns
df = df.drop(columns=['Fully equipped kitchen', 'Furnished'])

#TEST
#Deleting the column State of the building (Have to also try with deleting the 21643 rows)
#df = df.drop(columns='State of the building')

df_rent = df[df['To rent'] == True]

df_rent_appart = df_rent[(df_rent['type'] == 'Appartement') | (df_rent['type'] == 'Rez-de-chaussée')]

#df_rent_appart = df_rent_appart.dropna(subset="Living Area") #Deleting the empty cells of leaving area

df_rent_appart.isnull().sum()

df_rent_appart = df_rent_appart.drop(columns=['To rent', 'To sell', 'type', "Number of rooms"])

df_rent_appart = df_rent_appart.dropna(subset='State of the building')

df_rent_appart = df_rent_appart.dropna(subset="Living Area")

#df_rent_appart = df_rent_appart.dropna(subset="Number of rooms")


#Checking the numbers of empty cells
df_rent_appart.isnull().sum()
#df_rent_appart.shape


Price                    0
Living Area              0
Open fire                0
Terrace                  0
Garden                   0
Swimming pool            0
State of the building    0
zipcode                  0
dtype: int64

In [1332]:
#Creating the column "Province"

df_rent_appart[(df_rent_appart["zipcode"] >= 1000) & (df_rent_appart["zipcode"] <= 1299)]

Province = []
for zipcode in df_rent_appart['zipcode']:
    if (zipcode >= 1000) & (zipcode <= 1299): Province.append('Brussels')
    elif (zipcode >= 1300) & (zipcode <= 1499): Province.append('Brabant wallon')
    elif (zipcode >= 2000) & (zipcode <= 2999): Province.append('Anvers')
    elif (zipcode >= 3500) & (zipcode <= 3999): Province.append('Limbourg')
    elif (zipcode >= 4000) & (zipcode <= 4999): Province.append('Liege')
    elif (zipcode >= 5000) & (zipcode <= 5680): Province.append('Namur')
    elif (zipcode >= 6600) & (zipcode <= 6999): Province.append('Luxembourg')
    elif (zipcode >= 8000) & (zipcode <= 8999): Province.append('Flandre occiendentale')
    elif (zipcode >= 9000) & (zipcode <= 9999): Province.append('Flandre orientale')
    elif (zipcode >= 1500) & (zipcode <= 1999) or (zipcode >= 3000) & (zipcode <= 3499): Province.append('Brabant flamand')
    elif (zipcode >= 6000) & (zipcode <= 6599) or (zipcode >= 7000) & (zipcode <= 7999): Province.append('Hainaut')

df_rent_appart['Province'] = Province
del df_rent_appart['zipcode'] #Not needed anymore

In [1333]:
df_rent_appart.head(10)

,Price,Living Area,Open fire,Terrace,Garden,Swimming pool,State of the building,Province
1,790.0,140.0,False,False,False,False,\nBon\n,Brabant flamand
3,520.0,50.0,False,False,False,False,\nBon\n,Luxembourg
6,660.0,95.0,False,False,False,False,\nExcellentétat\n,Liege
7,925.0,100.0,False,True,False,False,\nFraîchementrénové\n,Brabant flamand
14,1650.0,84.0,False,True,False,False,\nExcellentétat\n,Anvers
15,1650.0,84.0,False,True,False,False,\nExcellentétat\n,Anvers
17,860.0,95.0,False,True,False,False,\nExcellentétat\n,Hainaut
30,950.0,80.0,False,False,False,False,\nExcellentétat\n,Brussels
33,810.0,108.0,False,True,False,False,\nExcellentétat\n,Liege
34,1350.0,56.0,False,True,False,False,\nExcellentétat\n,Anvers


In [1334]:
df_rent_appart.columns

Index(['Price', 'Living Area', 'Open fire', 'Terrace', 'Garden',
       'Swimming pool', 'State of the building', 'Province'],
      dtype='object')

In [1335]:
df_rent_appart = df_rent_appart[df_rent_appart['Price'] <= 8000] #Deleting the 120120 and 66000 price for renting !
df_rent_appart = df_rent_appart[df_rent_appart['Price'] >= 275] #Deleting the price 45 for renting 89 square m...

#df_rent_appart.nlargest(20, 'Living Area')

df_rent_appart = df_rent_appart.query("`Living Area` <= 550") #Deleting the Living Area > 550 square m
df_rent_appart.shape

(6662, 8)

In [1336]:
df_rent_appart = pd.get_dummies(data = df_rent_appart, columns = ['Province', 'State of the building'])

In [1337]:
#Modelization part for renting appart in Belgium with LinearRegression

#Defining the features and the target

print(df_rent_appart.columns)

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part

regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

Index(['Price', 'Living Area', 'Open fire', 'Terrace', 'Garden',
       'Swimming pool', 'Province_Anvers', 'Province_Brabant flamand',
       'Province_Brabant wallon', 'Province_Brussels',
       'Province_Flandre occiendentale', 'Province_Flandre orientale',
       'Province_Hainaut', 'Province_Liege', 'Province_Limbourg',
       'Province_Luxembourg', 'Province_Namur',
       'State of the building_\nBon\n',
       'State of the building_\nExcellentétat\n',
       'State of the building_\nFraîchementrénové\n',
       'State of the building_\nÀrafraîchir\n',
       'State of the building_\nÀrestaurer\n',
       'State of the building_\nÀrénover\n'],
      dtype='object')
The score of the model for training part : 0.629256029613949
The score of the model for testing part : 0.6671118333718611
